In [ ]:
#%matplotlib ipympl
import numpy as np
import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
def generate_defect_field(size=100, defect_type='+1/2', defect_location=(1, 1), defect_strength=0.1):
    x, y = np.meshgrid(np.arange(size), np.arange(size))
    field = np.zeros((size, size, 2))
    rotation = np.random.uniform(0,np.pi*2)
    # Generate a +1/2 defect
    if defect_type == '+1/2':
        charge = 0.5
        r = np.sqrt((x - defect_location[0]) ** 2 + (y - defect_location[1]) ** 2)
        theta = charge*np.arctan2(y - defect_location[1], x - defect_location[0]) + rotation
        field[:, :, 0] = np.sin(theta) * defect_strength
        field[:, :, 1] = -np.cos(theta) * defect_strength

    # Generate a -1/2 defect
    elif defect_type == '-1/2':
        charge = -0.5
        r = np.sqrt((x - defect_location[0]) ** 2 + (y - defect_location[1]) ** 2)
        theta = charge*np.arctan2(y - defect_location[1], x - defect_location[0]) + rotation
        field[:, :, 0] = -np.sin(theta) * defect_strength
        field[:, :, 1] = np.cos(theta) * defect_strength
    elif defect_type == 'none':
        theta = np.random.uniform(0,np.pi*2)
        field[:, :, 0] = np.sin(theta)
        field[:, :, 1] = -np.cos(theta)
    norm = np.sqrt(field[:,:,0]**2 + field[:,:,1]**2)
    field[:,:,0] = field[:,:,0]/norm
    field[:,:,1] = field[:,:,1]/norm
    return field

def plot_defect_field(field):
    size = field.shape[0]
    x, y = np.meshgrid(np.arange(size), np.arange(size))
    plt.quiver(x,y,field[:, :, 0], field[:, :, 1],
                   pivot='mid',
                   headlength=0.2,
                   headwidth=0.2,
                   headaxislength=0.1,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.75
    )
    plt.xlim(-1,3)
    plt.ylim(-1,3)
    plt.show()

def generate_labeled_dataset(num_samples=100, size=100, defect_strength=0.1):
    X = []
    y = []

    for _ in range(num_samples):
        # Randomly choose a defect type and location
        defect_type = np.random.choice(['+1/2', '-1/2','none'])
        defect_location = (1,1)#(np.random.randint(size), np.random.randint(size))
        # Generate the defect field
        defect_field = generate_defect_field(size=size, defect_type=defect_type, defect_location=defect_location, defect_strength=defect_strength)

        # Append to the dataset
        X.append(defect_field.flatten())
        if defect_type == "+1/2":
            y.append(1)
        elif defect_type == "-1/2":
            y.append(2)
        else:
            y.append(0)
        #y.append(1 if defect_type == '+1/2' else 0)

    return np.array(X), np.array(y)
                    
# Example usage:
num_samples = 5000
size = 3
defect_strength = 1#0.1

X, y = generate_labeled_dataset(num_samples=num_samples, size=size, defect_strength=defect_strength)

# Print labels to check the distribution of +1/2 and -1/2 defects
print("Label distribution:", np.unique(y, return_counts=True))

# Visualize a few examples
test_pos_field = generate_defect_field(size,"+1/2",(1,1),1)

# test_neg_field = generate_defect_field(size,"-1/2",(1,1),1)
# w = get_winding_number(test_neg_field,(1,1))
# print(w,"-1/2")
# w = calculate_winding_number(test_field)
# print(w)
        
for i in range(5):
    plot_defect_field(X[i].reshape((size, size, 2)))

In [ ]:
def get_winding_number(field,loc):
    """ returns a winding number in radians for the rotation of vectors around the defect core
    inputs:
            field: a 2d vector field [x,y,u,v] of director field
            loc: a tuple containing the defect location
    returns:
            w: winding number of defect located at 'loc' in radians
    """
    u = field[:,:,0]
    v = field[:,:,1]
    dx = [0,-1,-1,-1,0,1,1,1]
    dy = [1,1,0,-1,-1,-1,0,1]
    w = 0.0
    print(loc)
    v1 = np.zeros(2)
    v2 = np.zeros(2)
    v1[0] = u[loc[0] + dx[0],loc[1] + dy[0]]
    v1[1] = v[loc[0] + dx[0],loc[1] + dy[0]]
    print(0,
          (loc[0]+dx[0],loc[1]+dy[0]),
          np.rad2deg(np.arctan2(v1[1],v1[0])),
          np.rad2deg(np.arctan2(v2[1],v2[0])),
          np.rad2deg(np.arctan2(v1[1],v1[0]))-np.rad2deg(np.arctan2(v2[1],v2[0])))
    for iloc in range(1,len(dx)):
        v2[0] = u[loc[0] + dx[iloc],loc[1] + dy[iloc]]
        v2[1] = v[loc[0] + dx[iloc],loc[1] + dy[iloc]]
        print(iloc,
              (loc[0]+dx[iloc],loc[1]+dy[iloc]),
              np.rad2deg(np.arctan2(v1[1],v1[0])),
              np.rad2deg(np.arctan2(v2[1],v2[0])),
              np.rad2deg(np.arctan2(v1[1],v1[0]))-np.rad2deg(np.arctan2(v2[1],v2[0])))
        #w += np.arccos(np.clip(np.dot(v2, v1), -1.0, 1.0))
        print(w)
        w += np.rad2deg(np.arctan2(v1[1],v1[0]))-np.rad2deg(np.arctan2(v2[1],v2[0]))
        print(w)
        v1 = v2
    return w
print(test_pos_field.shape)
plot_defect_field(test_pos_field)
w = get_winding_number(test_pos_field,(1,1))
print(w,"+1/2")

In [ ]:
# Assuming you have X and y from the generate_labeled_dataset function

# Convert NumPy arrays to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Split the data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X_tensor, y_tensor, test_size=0.3, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define a simple neural network model
class DefectClassifier(nn.Module):
    def __init__(self, input_size):
        super(DefectClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 3) #output size changed to 3 for 3 classes [-1/2,+1/2,no defect]
        #self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        #x = self.sigmoid(x)
        x = self.softmax(x)
        return x

# Initialize the model, loss function, and optimizer
print(X_tensor.shape[1])
model = DefectClassifier(input_size=X_tensor.shape[1])
#criterion = nn.BCELoss() # criterion for just predicting +1/2, -1/2
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set up early stopping
best_model_state_dict = None
best_validation_loss = float('inf')
patience = 10 # Adjust this based on your scenario
counter = 0

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    #loss = criterion(outputs, y_train.view(-1, 1))
    loss = criterion(outputs, y_train.long())

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.16f}')

    # Calculate validation loss
    with torch.no_grad():
        model.eval()
        validation_outputs = model(X_validation)
        validation_loss = criterion(validation_outputs, y_validation.long())

    print(f'Validation Loss: {validation_loss.item():.16f} best_validation_loss: {best_validation_loss:.16f}')
    print(validation_loss.item(),best_validation_loss,np.abs(validation_loss-best_validation_loss))
    # Check for early stopping
    if validation_loss.item() <= best_validation_loss:
    #if np.isclose(np.abs(validation_loss-best_validation_loss),0.0,rtol=1e-15):
    #if np.isclose(validation_loss.item(), best_validation_loss, rtol=1e-15):
        best_validation_loss = validation_loss
        best_model_state_dict = model.state_dict()
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping after {epoch + 1} epochs.')
            break
    print(counter)

# Load the best model
model.load_state_dict(best_model_state_dict)

# Evaluate the model on the test set
with torch.no_grad():
    model.eval()
    # y_pred = (model(X_test) > 0.5).float()
    # print(classification_report(y_test.numpy(), y_pred.numpy()))
    outputs_validation = model(X_validation)
    predictions_validation = torch.argmax(outputs_validation,dim=1)
    accuracy = (predictions_validation == y_validation).float().mean().item()
print(f"Validation Accuracy: {accuracy:.8f}")
    


In [ ]:
# Assuming you have a single test field as a NumPy array called 'single_test_field'
defect_type = np.random.choice(['+1/2', '-1/2','none'])
test_field = generate_defect_field(size=3, 
                                    defect_type=defect_type,
                                    defect_location=(1,1),
                                    defect_strength=1.0)
print(defect_type)
test_tensor = torch.tensor(test_field.flatten(), dtype=torch.float32).unsqueeze(0)


# Make a prediction
with torch.no_grad():
    model.eval()
    output = model(test_tensor)
    prediction = torch.argmax(output,dim=1)

defect_types = ["No Defect", "+1/2 Defect", "-1/2 Defect"]
label = defect_types[prediction.item()]
print(prediction.item())
print("Prediction:", label)

In [ ]:
# here we're going to generate another dataset and test the model against it
num_samples = 1000
size = 3
defect_strength = 1#0.1

X2, y2 = generate_labeled_dataset(num_samples=num_samples, size=size, defect_strength=defect_strength)
X2_tensor = torch.tensor(X2, dtype=torch.float32)
y2_tensor = torch.tensor(y2, dtype=torch.float32)
with torch.no_grad():
    model.eval()
    # y_pred = (model(X_test) > 0.5).float()
    # print(classification_report(y_test.numpy(), y_pred.numpy()))
    outputs_validation = model(X2_tensor)
    predictions_validation = torch.argmax(outputs_validation,dim=1)
    accuracy = (predictions_validation == y2_tensor).float().mean().item()
print(f"Validation Accuracy: {accuracy:.8f}")

In [ ]:
# this tests the moving window over a size of 100,100 for defect identification
defect_type = np.random.choice(["+1/2","-1/2"])
defect_location = (np.random.randint(2,98),np.random.randint(2,98))
large_test_field = generate_defect_field(size=100,
                                         defect_type=defect_type,
                                         defect_location=defect_location,
                                         defect_strength=1.0)
print(defect_type)
print(defect_location)

def plot_defect_field(field,size,loc):
    x, y = np.meshgrid(np.arange(size), np.arange(size))
    plt.quiver(x,y,field[:, :, 0], field[:, :, 1],
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.75
    )
    # plt.quiver(x,y,field[loc[0]-1:loc[0]+2, loc[1]-1:loc[1]+2, 0], 
    #                field[loc[0]-1:loc[0]+2, loc[1]-1:loc[1]+2, 1],
    #                pivot='mid',
    #                headlength=0.2,
    #                headwidth=0.2,
    #                headaxislength=0.1,
    #                color="tab:blue",
    #                scale_units='xy',
    #                scale=0.75
    # )
    # plt.plot(loc[0],loc[1],color="tab:red",marker='o')
    plt.show()

loc = defect_location
xmin = loc[0]-1
xmax = loc[0]+2
ymin = loc[1]-1
ymax = loc[1]+2
defect_highlighted_x = np.lib.stride_tricks.sliding_window_view(large_test_field[:,:,0],(3,3))
defect_highlighted_y = np.lib.stride_tricks.sliding_window_view(large_test_field[:,:,1],(3,3))
print(defect_highlighted_x.shape)

test_x = loc[1]-1
test_y = loc[0]-1
print(defect_highlighted_x[test_x,test_y].shape)

#plot_defect_field(defect_highlighted,3,defect_location)
defect_test_field = np.zeros((3,3,2))
defect_test_field[:,:,0] = defect_highlighted_x[test_x,test_y]
defect_test_field[:,:,1] = defect_highlighted_y[test_x,test_y]
plot_defect_field(defect_test_field,3,loc)
test_tensor = torch.tensor(defect_test_field.flatten(), dtype=torch.float32).unsqueeze(0)
# Make a prediction
with torch.no_grad():
    model.eval()
    output = model(test_tensor)
    prediction = torch.argmax(output,dim=1)

defect_types = ["No Defect", "+1/2 Defect", "-1/2 Defect"]
label = defect_types[prediction.item()]
print(prediction.item())
print("Prediction:", label)



defects_found = 0
defect_locations = []
for i in range(defect_highlighted_x.shape[0]):
    for j in range(defect_highlighted_y.shape[1]):
        defect_highlighted_field = np.zeros((3,3,2))
        defect_highlighted_field[:,:,0] = defect_highlighted_x[i,j]
        defect_highlighted_field[:,:,1] = defect_highlighted_y[i,j]
        test_tensor = torch.tensor(defect_highlighted_field.flatten(), dtype=torch.float32).unsqueeze(0)

        # Make a prediction
        with torch.no_grad():
            model.eval()
            output = model(test_tensor)
            prediction = torch.argmax(output,dim=1)
        defect_types = ["No Defect", "+1/2 Defect", "-1/2 Defect"]
        label = defect_types[prediction.item()]
        if prediction.item() > 0:
            print("Prediction:", label,prediction.item(),"loc:",j+1,i+1)
            defect_locations.append([j+1,i+1])
            defects_found += 1
print(defects_found)

x, y = np.meshgrid(np.arange(100), np.arange(100))
plt.quiver(x,y,large_test_field[:, :, 0], large_test_field[:, :, 1],
               pivot='mid',
               headlength=0,
               headwidth=0,
               headaxislength=0,
               color="tab:blue",
               scale_units='xy',
               scale=0.75
)
defect_locations = np.array(defect_locations)
plt.plot(defect_locations[:,0],defect_locations[:,1],color="tab:red",marker=",")
plt.show()